#Predicting Taxi Ride Durations

## Setup Spark in Google Colab
*reference: https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/*


*to install other versions, get the download link from https://spark.apache.org/downloads.html*

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

--2022-12-11 23:48:07--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz.2’

spark-3.3.1-bin-had 100%[===================>] 285.48M   153MB/s    in 1.9s    

2022-12-11 23:48:09 (153 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz.2’ saved [299350810/299350810]



In [ ]:
!tar -xvf spark-3.3.1-bin-hadoop3.tgz

spark-3.3.1-bin-hadoop3/
spark-3.3.1-bin-hadoop3/LICENSE
spark-3.3.1-bin-hadoop3/NOTICE
spark-3.3.1-bin-hadoop3/R/
spark-3.3.1-bin-hadoop3/R/lib/
spark-3.3.1-bin-hadoop3/R/lib/SparkR/
spark-3.3.1-bin-hadoop3/R/lib/SparkR/DESCRIPTION
spark-3.3.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/Rd.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/features.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/hsearch.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/links.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/nsInfo.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/package.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/Meta/vignette.rds
spark-3.3.1-bin-hadoop3/R/lib/SparkR/NAMESPACE
spark-3.3.1-bin-hadoop3/R/lib/SparkR/R/
spark-3.3.1-bin-hadoop3/R/lib/SparkR/R/SparkR
spark-3.3.1-bin-hadoop3/R/lib/SparkR/R/SparkR.rdb
spark-3.3.1-bin-hadoop3/R/lib/SparkR/R/SparkR.rdx
spark-3.3.1-bin-hadoop3/R/lib/SparkR/doc/
spark-3.3.1-bin-hadoop3/R/lib/Spar

In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

#Exploratory Data Analysis

visualization of all our data, taken from train.csv of the 2016 NYC Taxi Dataset

https://www.kaggle.com/code/headsortails/nyc-taxi-eda-update-the-fast-the-curious#a-simple-model-and-prediction

In [ ]:
#EDA

dfTaxi = spark.read.csv('train.csv', inferSchema=True, header=True)
print('Column Names and Type: ')
dfTaxi.printSchema()

Column Names and Type: 
root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: integer (nullable = true)



In [ ]:
#sample data points

print('Showing First 3 Rows: ')
dfTaxi.show(3)

Showing First 3 Rows: 
+---------+---------+-------------------+-------------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:00|2016-03-14 17:32:00|              1|    -73.98215485|    40.76793671|     -73.96463013|     40.76560211|                 N|          455|
|id2377394|        1|2016-06-12 00:43:00|2016-06-12 00:54:00|              1|    -73.98041534|    40.73856354|      -73.9994812|     40.73115158|                 N|          663|
|id3858529|        2|2016-01-19 11:35:00|2016-01-19 12:10:00|              1|    -

In [ ]:
#describe

dfTaxi.describe().show()

+-------+---------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|       id|         vendor_id|   passenger_count|   pickup_longitude|    pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|    trip_duration|
+-------+---------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|  1048575|           1048575|           1048575|            1048575|            1048575|           1048575|           1048575|           1048575|          1048575|
|   mean|     null| 1.534502539160289|1.6643816608254058|  -73.9734195748197| 40.750942000040084|-73.97336114636339| 40.75183426921277|              null|962.1447841117707|
| stddev|     null|0.4988083921459773| 1.314260596411729|0.04280165136265032|0.03381389424267591|0.0427428230329855|0.0364500232351455|

In [ ]:
#store trip duration stddev and mean to remove outliers
from pyspark.sql.functions import stddev
from pyspark.sql.functions import mean

trip_stddev = dfTaxi.select(stddev('trip_duration')).show()
trip_mean = dfTaxi.select(mean('trip_duration')).show()


+--------------------------+
|stddev_samp(trip_duration)|
+--------------------------+
|         5853.002177831257|
+--------------------------+

+------------------+
|avg(trip_duration)|
+------------------+
| 962.1447841117707|
+------------------+



In [ ]:
#check null and empty values

for column in dfTaxi.columns:
  print(column, ":", dfTaxi.filter(dfTaxi[column].isNull()).count(), "null values")

id : 0 null values
vendor_id : 0 null values
pickup_datetime : 0 null values
dropoff_datetime : 0 null values
passenger_count : 0 null values
pickup_longitude : 0 null values
pickup_latitude : 0 null values
dropoff_longitude : 0 null values
dropoff_latitude : 0 null values
store_and_fwd_flag : 0 null values
trip_duration : 0 null values


#Preprocessing

transformations that our RDD's will go through such that it is usable for our ML models

In [ ]:
sc = spark.sparkContext

In [ ]:
#load dataset from train.csv into Taxi RDD
from traitlets.traitlets import ForwardDeclaredInstance
TaxiRDD = sc.textFile('train.csv',8)


In [ ]:
print('Number of Data Points: ', TaxiRDD.count())
print('Column Names: ')
print(TaxiRDD.take(1))

Number of Data Points:  1048576
Column Names: 
['id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration']


##Remove Header

we extract the header from the dataset, leaving only data records

In [ ]:
def filterHeader(TaxiRecords):
    """ Skips lines with header information
    Args:
        TaxiRecord : a line from the taxi file.

    Returns:
        True - if line is not header; False otherwise
    """
    fieldsList = TaxiRecords.split(',')
    if  'trip_duration' in fieldsList[-1]:
        return False
    else:
        return True

filteredTaxiRDD = TaxiRDD.filter(filterHeader)
print(filteredTaxiRDD.take(5))

['id2875421,2,2016-03-14 17:24,2016-03-14 17:32,1,-73.98215485,40.76793671,-73.96463013,40.76560211,N,455', 'id2377394,1,2016-06-12 0:43,2016-06-12 0:54,1,-73.98041534,40.73856354,-73.9994812,40.73115158,N,663', 'id3858529,2,2016-01-19 11:35,2016-01-19 12:10,1,-73.97902679,40.7639389,-74.00533295,40.71008682,N,2124', 'id3504673,2,2016-04-06 19:32,2016-04-06 19:39,1,-74.01004028,40.7199707,-74.01226807,40.70671844,N,429', 'id2181028,2,2016-03-26 13:30,2016-03-26 13:38,1,-73.97305298,40.79320908,-73.97292328,40.78252029,N,435']


## Remove Outliers

from EDA we can see that same data records include very large values for trip_duration

 we will handle this by only using values within one std from the mean

In [ ]:
#using STD taken from EDA where std = 5853 and mean is 962
def removeOutliers(filteredRecords):
    """ Removes all data which is above one standard deviation from the mean
    Args:
        filteredRecord : a line from the taxi file.

    Returns:
        True - if line is  outlier return False, True otherwise
    """
    stddev = 5853
    mean = 962
    fieldsList = filteredRecords.split(',')
    if  int(fieldsList[-1]) > stddev + mean:
        return False
    else:
        return True

filterOutlierTaxiRDD = filteredTaxiRDD.filter(removeOutliers)
print(filteredTaxiRDD.count())
print(filterOutlierTaxiRDD.count())

1048575
1046915


##Extract id from records

since id is unique for each record, it will not be important in creation of models

In [ ]:
def extractID(filTaxiRecord):
    """Extract id field, leaving all other remaining features
    Args:
        filTaxiRecord : a comma separated string consisting of all fields in the data set.

    Returns:
        extracted record: a list of all features excluding id
    """
    fieldsList = filTaxiRecord.split(',')
    return (fieldsList[1:])

noIDTaxiRDD = filterOutlierTaxiRDD.map(extractID)
print(noIDTaxiRDD.take(2))


[['2', '2016-03-14 17:24', '2016-03-14 17:32', '1', '-73.98215485', '40.76793671', '-73.96463013', '40.76560211', 'N', '455'], ['1', '2016-06-12 0:43', '2016-06-12 0:54', '1', '-73.98041534', '40.73856354', '-73.9994812', '40.73115158', 'N', '663']]


## Fix datetime format

expand datetime features into hour, day, and hour features

year will be excluded as it is the same for each record, and minutes is used to round up the hour

In [ ]:
def fixDate(noIDTaxiRecord):
    """Creates a key-value pair consisting of the Date field as the key and the the tuple (Day,Commute Time at 5 PM, Total number of Accidents) as the value

    Args:
        deerfootRDDRecord : a comma separated string consisting of all fields in the data set.

    Returns:
        extracted record: key-value pair as detailed above
    """
    year1, month1, day1 = noIDTaxiRecord[1].split('-')
    day1, hour1 = day1.split(' ')
    hour1, min1 = hour1.split(':')
    if int(min1) >= 30:
      hour1 = int(hour1)
      if hour1 != 24:
        hour1 += 1
      else:
        hour1 = 0
    list1 = [month1, day1, hour1]

    year2, month2, day2 = noIDTaxiRecord[2].split('-')
    day2, hour2 = day2.split(' ')
    hour2, min2 = hour2.split(':')
    if int(min2) >= 30:
      hour2 = int(hour2)
      if hour2 != 24:
        hour2 += 1
      else:
        hour2 = 0
    list2 = [month2, day2, hour2]

    return (noIDTaxiRecord[0:1] + list1 + list2 + noIDTaxiRecord[3:])

fixedDateTaxiRDD = noIDTaxiRDD.map(fixDate)
print(fixedDateTaxiRDD.take(1))


[['2', '03', '14', '17', '03', '14', 18, '1', '-73.98215485', '40.76793671', '-73.96463013', '40.76560211', 'N', '455']]


## Encode store_and_fwd_flag

convert Y to 1 and N to 0 for the store_and_fwd feature

In [ ]:
def fixStoreForward(dateTaxiRecord):
    """Creates a key-value pair consisting of the Date field as the key and the the tuple (Day,Commute Time at 5 PM, Total number of Accidents) as the value

    Args:
        deerfootRDDRecord : a comma separated string consisting of all fields in the data set.

    Returns:
        extracted record: key-value pair as detailed above
    """
    if dateTaxiRecord[12] == 'Y':
      dateTaxiRecord[12] = 1
    if dateTaxiRecord[12] == 'N':
      dateTaxiRecord[12] = 0

    return (dateTaxiRecord)

fixedTaxiRDD = fixedDateTaxiRDD.map(fixStoreForward)
print(fixedTaxiRDD.take(2))

[['2', '03', '14', '17', '03', '14', 18, '1', '-73.98215485', '40.76793671', '-73.96463013', '40.76560211', 0, '455'], ['1', '06', '12', 1, '06', '12', 1, '1', '-73.98041534', '40.73856354', '-73.9994812', '40.73115158', 0, '663']]


##Convert to LabeledPoint RDD

convert each record into a LabeledPoint format

In [ ]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

def labelPoint(label_features):
    """Converts a comma separated unicode string into a `LabeledPoint`.

    Args:
        line (unicode): Comma separated unicode string where the last element is the label and the
            remaining elements are features.

    Returns:
        LabeledPoint: The line is converted into a `LabeledPoint`, which consists of a label and
            features.
    """
    ret_val = LabeledPoint(label_features[-1],label_features[0:-1])
    return ret_val

labeledPointRDD = fixedTaxiRDD.map(labelPoint)
firstPoint = labeledPointRDD.take(1)
firstPointFeatures =firstPoint[0].features 
firstPointLabel = firstPoint[0].label
print (firstPointFeatures, firstPointLabel)

[2.0,3.0,14.0,17.0,3.0,14.0,18.0,1.0,-73.98215485,40.76793671,-73.96463013,40.76560211,0.0] 455.0


##Normalize Features

features are normalized using the function found in the ML_Linear_Regression.ipynb by Sarah Shah

In [ ]:
def normalizeFeatures(lp):
    """Normalizes features in the LabeledPoint object lp.

    Args:
        lp - LabeledPoint object 

    Returns:
        LabeledPoint: The object contains the label and the normalized features
    """
    normalizedFeatures = list()
    for i in range(0,len(lp.features)):
        feature = (lp.features[i]-broadcastMean.value[i])/broadcastStdev.value[i]
        normalizedFeatures.insert(i,feature)
    return LabeledPoint(lp.label, normalizedFeatures)


def getNormalizedRDD(nonNormalizedRDD): 
    """Normalizes the features of the LabeldPoints contained in nonNormalizedRDD.

    Args:
        nonNormalizedRDD - RDD containing non-normalized features 

    Returns:
        returnRDD: RDD containing normalized features
    """


    meanList = list()
    stdevList = list()
    numFeatures = len(nonNormalizedRDD.take(1)[0].features)
    for i in range(0,numFeatures):
        featureRDD = nonNormalizedRDD.map(lambda lp: lp.features[i])
        featureMean = featureRDD.mean()
        featureStdev = featureRDD.stdev()
        meanList.insert(i,featureMean)
        stdevList.insert(i,featureStdev)
    global broadcastMean 
    broadcastMean = sc.broadcast(meanList)
    global broadcastStdev 
    broadcastStdev = sc.broadcast(stdevList)
    returnRDD = nonNormalizedRDD.map(normalizeFeatures)
    return returnRDD

normalizedSamplePoints = getNormalizedRDD(labeledPointRDD)
print(normalizedSamplePoints.take(5))

[LabeledPoint(455.0, [0.9344700074143757,-0.3068418913295311,-0.17215291368394903,0.450100955763032,-0.30699567527148675,-0.17219497155044736,0.5989693833015522,-0.5053151399659359,-0.2039363282189441,0.5025011382908023,0.20465235924955982,0.3777923028783001,-0.07468407754132475]), LabeledPoint(663.0, [-1.0701253031832862,1.478111009962675,-0.4019430606254322,-2.042696732996786,1.4777642492997434,-0.401984858978578,-2.0153179424934944,-0.5053151399659359,-0.16323684191552718,-0.36670320054947686,-0.6115417300413228,-0.5678057241885891,-0.07468407754132475]), LabeledPoint(2124.0, [0.9344700074143757,-1.496810492191002,0.40232245366975905,-0.32889832197441105,-1.49683562498564,0.4022797470198793,-0.32372026109669955,-0.5053151399659359,-0.1307488011995465,0.3841988305092362,-0.748586697728116,-1.145991183066537,-0.07468407754132475]), LabeledPoint(429.0, [0.9344700074143757,0.2881424091012043,-1.0913135014498818,0.9175005224054978,0.28792429958558996,-1.09135452126297,0.9065325981009695,

#Model Creation

In [ ]:
# train test split

weight = [.8 , .2]
train, test = normalizedSamplePoints.randomSplit(weight, seed = 42)
train.cache()
test.cache()

print("Total Count: ", normalizedSamplePoints.count())
print("Train Count: ", train.count())
print("Test Count: ", test.count())


Total Count:  1046915
Train Count:  838012
Test Count:  208903


In [ ]:
print(train.take(5))

[LabeledPoint(455.0, [0.9344700074143757,-0.3068418913295311,-0.17215291368394903,0.450100955763032,-0.30699567527148675,-0.17219497155044736,0.5989693833015522,-0.5053151399659359,-0.2039363282189441,0.5025011382908023,0.20465235924955982,0.3777923028783001,-0.07468407754132475]), LabeledPoint(663.0, [-1.0701253031832862,1.478111009962675,-0.4019430606254322,-2.042696732996786,1.4777642492997434,-0.401984858978578,-2.0153179424934944,-0.5053151399659359,-0.16323684191552718,-0.36670320054947686,-0.6115417300413228,-0.5678057241885891,-0.07468407754132475]), LabeledPoint(2124.0, [0.9344700074143757,-1.496810492191002,0.40232245366975905,-0.32889832197441105,-1.49683562498564,0.4022797470198793,-0.32372026109669955,-0.5053151399659359,-0.1307488011995465,0.3841988305092362,-0.748586697728116,-1.145991183066537,-0.07468407754132475]), LabeledPoint(429.0, [0.9344700074143757,0.2881424091012043,-1.0913135014498818,0.9175005224054978,0.28792429958558996,-1.09135452126297,0.9065325981009695,

Imported the RMSE function found in the ML_Linear_Regression.ipynb by Sarah Shah

In [ ]:
import math
def squaredError(label, prediction):
    """Calculates the the squared error for a single prediction.

    Args:
        label (float): The correct value for this observation.
        prediction (float): The predicted value for this observation.

    Returns:
        float: The difference between the `label` and `prediction` squared.
    """
    sqrError = (label-prediction)*(label-prediction)
    return sqrError

def calcRMSE(labelsAndPreds):
    """Calculates the root mean squared error for an `RDD` of (label, prediction) tuples.

    Args:
        labelsAndPred (RDD of (float, float)): An `RDD` consisting of (label, prediction) tuples.

    Returns:
        float: The square root of the mean of the squared errors.
    """
    sqrSum = labelsAndPreds.map(lambda s: squaredError(s[0],s[1])).sum()
    return math.sqrt(sqrSum/labelsAndPreds.count())

##Baseline Model
using the mean of trip_duration we determine a baseline model which will be used to compare our other models

In [ ]:
averageTripDur = (train.map(lambda s: s.label)).mean()
print("Mean:", averageTripDur)

labelsAndMean = test.map(lambda lp: (lp.label, averageTripDur))
print("RMSE for Baseline:", calcRMSE(labelsAndMean))

Mean: 835.2737514498589
RMSE for Baseline: 656.3879217129335


#Linear Regression Model

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD


lr = LinearRegressionWithSGD.train(train, iterations=500, step = 1, miniBatchFraction = 1, regType = 'l2', intercept = True)

labelAndPredTrain = train.map(lambda lp: (lp.label,lr.predict(lp.features)))

labelAndPredTest = test.map(lambda lp: (lp.label,lr.predict(lp.features)))

print("RMSE for LR train: ",calcRMSE(labelAndPredTrain))
print("RMSE for LR test: ",calcRMSE(labelAndPredTest))

/content/spark-3.3.1-bin-hadoop3/python/pyspark/mllib/regression.py:367: FutureWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.", FutureWarning)


RMSE for LR train:  604.1849277161394
RMSE for LR test:  600.8950026082762


#Ridge Regression Model

In [ ]:
from pyspark.mllib.regression import RidgeRegressionWithSGD


rr = RidgeRegressionWithSGD.train(train, iterations = 500, step = 1, miniBatchFraction = 1, intercept = True)

labelAndPredTrain = train.map(lambda lp: (lp.label,rr.predict(lp.features)))

labelAndPredTest = test.map(lambda lp: (lp.label,rr.predict(lp.features)))

print("RMSE for LR train: ",calcRMSE(labelAndPredTrain))
print("RMSE for LR test: ",calcRMSE(labelAndPredTest))

/content/spark-3.3.1-bin-hadoop3/python/pyspark/mllib/regression.py:723: FutureWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression with elasticNetParam = 0.0. Note the default regParam is 0.01 for RidgeRegressionWithSGD, but is 0.0 for LinearRegression.
  warnings.warn(


RMSE for LR train:  604.2477664442976
RMSE for LR test:  601.0435979272848


#Decision Tree Regressor Model

In [ ]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainRegressor(train, {}, impurity='variance', maxDepth=10, maxBins = 32)


predictionsTrain = model.predict(train.map(lambda lp: lp.features)).collect()
labelsTrain = train.map(lambda lp: lp.label).collect()
rmseDTTrain = np.sqrt(np.mean((np.array(predictionsTrain)-np.array(labelsTrain))**2))

predictions = model.predict(test.map(lambda lp: lp.features)).collect()
labels = test.map(lambda lp: lp.label).collect()
rmseDT = np.sqrt(np.mean((np.array(predictions)-np.array(labels))**2))

print("Training RMSE for DT: ", rmseDTTrain)
print("Testing RMSE for DT: ", rmseDT)

Training RMSE for DT:  431.69139843381157
Testing RMSE for DT:  435.2348271966641


#Random Forest Regressor Model

In [ ]:
from pyspark.mllib.tree import RandomForest

rf_model = RandomForest.trainRegressor(train, categoricalFeaturesInfo={}, numTrees=13, impurity='variance', maxDepth= 10, maxBins= 32)

predictionsTrain = rf_model.predict(train.map(lambda lp: lp.features)).collect()
labelsTrain = train.map(lambda lp: lp.label).collect()
rmseRFTrain = np.sqrt(np.mean((np.array(predictionsTrain)-np.array(labelsTrain))**2))

predictions = rf_model.predict(test.map(lambda lp: lp.features)).collect()
labels = test.map(lambda lp: lp.label).collect()
rmseRF = np.sqrt(np.mean((np.array(predictions)-np.array(labels))**2))


print("Training RMSE for RF: ", rmseRFTrain)
print("Testing RMSE for RF: ", rmseRF)

Training RMSE for RF:  418.9219831463758
Testing RMSE for RF:  421.6070828981862


#Gradient Boosted Trees Model

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees

gb_model = GradientBoostedTrees.trainRegressor(train, categoricalFeaturesInfo={}, maxDepth = 5, learningRate= 0.5)


In [ ]:
predictionsTrain = gb_model.predict(train.map(lambda lp: lp.features)).collect()
labelsTrain = train.map(lambda lp: lp.label).collect()
rmseGBTrain = np.sqrt(np.mean((np.array(predictionsTrain)-np.array(labelsTrain))**2))

predictions = gb_model.predict(test.map(lambda lp: lp.features)).collect()
labels = test.map(lambda lp: lp.label).collect()
rmseGB = np.sqrt(np.mean((np.array(predictions)-np.array(labels))**2))


print("Training RMSE for GB: ", rmseGBTrain)
print("Testing RMSE for GB: ", rmseGB)

Training RMSE for GB:  307.63101677876966
Testing RMSE for GB:  312.3534203126632
